In [4]:
import os
import sys

if 'has_changed_dir' not in globals():
    repo_path = os.path.abspath(os.path.join('..'))
    
    if repo_path not in sys.path:
        sys.path.append(repo_path)
    
    os.chdir(repo_path)
    
    globals()['has_changed_dir'] = True
    print(repo_path)
    print(os.getcwd())


from typing import Optional, Union

import torch
from transformers import AutoModelForSeq2SeqLM, M2M100Config, NllbTokenizer  # noqa: F401

from src.train_model import LightningModel




### Load PyTorch Lightning checkpoint

In [11]:
# Default model with updated embeddings and tokenizer
model_config = "models/re-init/config.json"
model_path: str = "models/re-init/"
tokenizer_path: str = "models/re-init/"
tokenizer_vocab_file_path: str = "models/re-init/sentencepiece.bpe.model"

# Updated weights to default model with updated embeddings and tokenizer
ckpt_path: Optional[Union[str, None]] = "models/it_1/epoch=17-val_loss=1.48184.ckpt"

# Converted model path
converted_model_path = "models/final"

In [6]:
# Load model using checkpoint:
# Option 1:
# model = AutoModelForSeq2SeqLM.from_config(config = M2M100Config.from_json_file(model_config))
# Option 2:
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
tokenizer = NllbTokenizer.from_pretrained(tokenizer_path, vocab_file = tokenizer_vocab_file_path)


# Option 1:
lightining_model = LightningModel.load_from_checkpoint(ckpt_path, map_location=torch.device("cuda:1"), model = model, tokenizer = tokenizer)

# Option 2:
# ckpt = torch.load(ckpt_path, map_location=torch.device("cuda:1"))
# lightining_model = LightningModel(model, tokenizer)
# lightining_model.load_state_dict(ckpt['state_dict'])

### Convert PyTorch Lightning checkpoint to Transformers

In [7]:
# Convert to Transformers
lightining_model.convert_ckpt_to_tranformers(converted_model_path)

del lightining_model, model, tokenizer

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200}


In [13]:
model = AutoModelForSeq2SeqLM.from_pretrained(converted_model_path)
tokenizer = NllbTokenizer.from_pretrained(converted_model_path, vocab_file = converted_model_path + "/sentencepiece.bpe.model")

lightining_model = LightningModel(model, tokenizer)

### Infer model

In [14]:
sentence: str = "Когда Исо услышал это, Он сказал: – Не здоровым нужен врач, а больным."

translation = lightining_model.predict(sentence, src_lang='rus_Cyrl' , tgt_lang='lez_Cyrl')

print(translation)

# Реальный перевод: И гафар ван хьайила, Исади абуруз лагьана: – Жерягь сагъбуруз ваъ, начагъбуруз герек я.

['И гафарин ван хьайила, Исади абуруз лагьана: - Жерягь сагъбуруз ваъ, начагъбуруз герек я.']
